### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [ ]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (f'{url}api-key={nyt_api_key}&begin_date={begin_date}'
             + f'&end_date={end_date}&fq={filter_query}&sort={sort}'
             + f'&fl={field_list}')

In [ ]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):
    # create query with a page number
    # API results show 10 articles at a time
    page_num = f'&page={page}'
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(f"{query_url}{page_num}").json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f'Checked page {page}')

        # Print the page number that had no results then break from the loop
    except:
        print(f'No results found for page {page}')

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
### COULD NOT USE " Review" because What's love got to do with it starts with 
### Review:.  Used \u2019 with a space after as alternative.
reviews_df['title'] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 ")])

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_df['title'].drop_duplicates()
# titles_list = reviews_df['title']
titles = titles_list.to_list()

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response_data = requests.get(f"{url}{title}{tmdb_key_string}").json()
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = response_data["results"][0]["id"]

        # Make a request for a the full movie details
        query_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        
        # Execute "GET" request with url
        movie_data = requests.get(query_url).json()
        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_data.get('genres', [])]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['name'] for lang in movie_data.get('spoken_languages', [])]

        # Extract the production_countries' name into a list
        production_countries = [countries['name'] for countries in movie_data.get('production_countries', [])]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({"title": movie_data["title"],
                     "original_title": movie_data["original_title"],
                     "budget": movie_data["budget"],
                     "language": movie_data["original_language"],
                     "homepage": movie_data["homepage"],
                     "overview": movie_data["overview"],
                     "popularity": movie_data["popularity"],
                     "runtime": movie_data["runtime"],
                     "revenue": movie_data["revenue"],
                     "release_date": movie_data["release_date"],
                     "vote_average": movie_data["vote_average"],
                     "vote_count": movie_data["vote_count"],
                     "genre": genres, "spoken_languages": spoken_languages,
                     "production_countries": production_countries})
        
        # Print out the title that was found
        print(f"{title} was found! Yay.")
    except:
        print(f"{title} was not found. Womp Womp.")


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list, indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
movie_review_df = pd.merge(reviews_df, tmdb_df, on="title")

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres','spoken_languages','production_countries']

# Create a list of characters to remove
characters_to_remove = ['[',']',"'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    movie_review_df = movie_review_df.astype({column:str}, errors='raise')

    # Loop through characters to remove
    for character in characters_to_remove:
        movie_review_df[column] = movie_review_df[column].str.replace(character,"")

# Display the fixed DataFrame
movie_review_df.head(1)

In [ ]:
# Drop "byline.person" column
movie_review_df = movie_review_df.drop("byline.person", axis=1)

In [ ]:
# Delete duplicate rows and reset index
movie_review_df = movie_review_df.drop_duplicates()
movie_review_df = movie_review_df.reset_index(drop=True)

In [ ]:
# Export data to CSV without the index
movie_review_df.to_csv("output/collected_data.csv", index=False, header=True)